In [1]:
import pandas as pd
import spacy,re,json
from flask import Flask, render_template, request
import pandas as pd
from pprint import pprint
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

# import NER model

In [2]:
ner_model = spacy.load('ner_model_Skills_me/')

# import Skill Hierarchical  relation

In [3]:
SKILL_Hierarchical_relation_DF = pd.read_excel(r'Hierarchical_relation_DF.xls')
SKILL_Hierarchical_relation_DF = SKILL_Hierarchical_relation_DF.drop_duplicates()
SKILL_Hierarchical_relation_DF

,node1,node2
0,data,Extract
1,Tableau,data visualization
2,Qlik,data visualization
3,algorithms,Deep learning
4,analytics,support
...,...,...
1174,machine learning,data
1187,statistics,data analytics
1189,design,A/B testing
1222,CSS3,CSS


# import Skill Correlation relation

In [4]:
# SKILL_Correlation_relation_DF = pd.read_excel(r'Correlation_relation_Link.xls')
# SKILL_Correlation_relation_DF = SKILL_Correlation_relation_DF.drop_duplicates()
# SKILL_Correlation_relation_DF

In [5]:
def get_HTML(my_doc, ent_list):
    total_html = ''
    for tem in my_doc:
        if  r'/r' in tem.text or  r'/n' in tem.text or  '\\r' in tem.text or '\\n' in tem.text:
            total_html += '<br/>'

        if tem not in ent_list:
            total_html += tem.text + ' '
        else:
            total_html += "<span style='background:yellow'>"+ tem.text +' ' + "</span>"

    display(HTML( '<html>'+ total_html + '</html>' ))
    return '<html>'+ total_html + '</html>'

In [6]:
def get_HTML_JD(my_doc, ent_list, cv_ent_list):
    total_html = ''
    for tem in my_doc:
        if  r'/r' in tem.text or  r'/n' in tem.text or  '\\r' in tem.text or '\\n' in tem.text:
            total_html += '<br/>'

        if tem not in ent_list:
            total_html += tem.text + ' '
        elif tem.text.lower() not in cv_ent_list:
            total_html += "<span style='background:red'>"+ tem.text +' ' + "</span>"
        else:
            total_html += "<span style='background:yellow'>"+ tem.text +' ' + "</span>"

    display(HTML( '<html>'+ total_html + '</html>' ))
    return '<html>'+ total_html + '</html>'

# Skill relation Dict

In [7]:
SKILL_relation_Dict = {}

for ind, row in SKILL_Hierarchical_relation_DF.iterrows():
    SKILL_relation_Dict[row['node1'].lower()] = row['node2'].lower()
    
    
# for ind, row in SKILL_Correlation_relation_DF.iterrows():
#     SKILL_relation_Dict[row['node1']] = row['node2']
    
SKILL_relation_Dict

{'data': 'data consumption',
 'tableau': 'data',
 'qlik': 'tableau',
 'algorithms': 'data exploration',
 'analytics': 'architecture',
 'software': 'dashboards',
 'statistical modeling': 'data',
 'mechanics': 'research',
 'relational databases': 'data',
 'sql server': 'ms office',
 'outlook': 'microsoft office',
 'excel': 'ms office',
 'word': 'ms office',
 'powerpoint': 'ms office',
 'python': 'data consumption',
 'pandas': 'analytics',
 'mongodb': 'neo4j',
 'sql': 'data analysis',
 'statistics': 'data analytics',
 'predictive modeling': 'data',
 'machine learning': 'data',
 'optimization': 'dashboards',
 'hypothesis testing': 'hypothesis testing',
 'r': 'data consumption',
 'matlab': 'scripting',
 'spotfire': 'data visualization',
 'tensorflow': 'deep learning',
 'keras': 'deep learning',
 'architecture': 'minimum',
 'analysis': 'advanced analytics',
 'python/r': 'software',
 'testing': 'business analytics',
 'spark': 'big data',
 'h2o': 'learning',
 'mysql': 'sql server',
 'neural ne

In [ ]:
app = Flask(__name__)

@app.route('/get_gap/', methods=['POST'])
# 返回最接近的向量
def get_simmilar():
    print( 'get_gap:' )

    JD = eval(request.get_data())['JD']
    CV = eval(request.get_data())['CV']
    
    ## skill word in JD
    doc1 = ner_model(JD)
    JD_tem_list = []
    for chunk in doc1.ents:
        JD_tem_list.append( chunk.text.lower() )
    JD_tem_list = list(set(JD_tem_list))

    JD_ent_list = [t.root for t in doc1.ents]
    
    ## skill word in CV
    doc2 = ner_model(CV)
    CV_tem_list = []
    for chunk in doc2.ents:
        CV_tem_list.append( chunk.text.lower() )
    CV_tem_list = list(set(CV_tem_list))

    CV_tem_list_expand = []
    for tem in CV_tem_list:
        CV_tem_list_expand.append( str(tem) )
        if SKILL_relation_Dict.get( str(tem),None ) != None:
            CV_tem_list_expand.append( SKILL_relation_Dict.get( str(tem),None ) )

    CV_tem_list_expand = list(set(CV_tem_list_expand))
    CV_ent_list = [t.root for t in doc2.ents]

    CV_tem_list_expand = [w.lower() for w in CV_tem_list_expand]
    
    JD_HTML = get_HTML_JD( doc1, JD_ent_list, CV_tem_list_expand)
    print( 'skill word in JD:', JD_tem_list)
    
    print( '=' * 100)
    CV_HTML = get_HTML( doc2, CV_ent_list)
    print( 'skill word in CV:', CV_tem_list_expand)
    
    gap_tem_list = []
    for t1 in JD_tem_list:
        if t1.lower() not in CV_tem_list_expand:
            gap_tem_list.append( t1 )
    
    print( '=' * 100)
    print( 'gap_list', gap_tem_list)
    response = { 
        'JD_HTML': JD_HTML,
        'CV_HTML': CV_HTML,
        'gap_list': gap_tem_list}
    
    return str(response)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5553)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5553/ (Press CTRL+C to quit)


get_gap:


skill word in JD: ['software platforms', 'web user interface design', 'design experience', 'object-oriented design', 'ui', 'programming languages', 'api', 'javascript', 'web programming skills', 'web applications', 'according', 'web services', 'css3', 'jquery', 'strive', 'verbal communication', 'navigation', 'executive management', 'soap', 'multimedia content development', 'html5', 'css', 'html', 'rest', 'teamwork']


127.0.0.1 - - [28/Sep/2020 05:40:48] "POST /get_gap/ HTTP/1.1" 200 -


...

127.0.0.1 - - [28/Sep/2020 05:54:48] "POST /get_gap/ HTTP/1.1" 200 -


skill word in CV: ['sql', 'j2ee', 'servlets', 'data analysis', 'analyzing', 'records', 'java', 'eap.java', 'javascript', 'testing and', 'j2ee designs', 'ajax', 'javascripts', 'data', 'jsps', 'bachelors', 'reviewing', 'software development', 'experience.•', 'jdbc', 'agile', 'computer science', 'masters degree', 'object oriented design', 'r', 'communication skills', 'taking', 'class diagrams', 'received', 'mysql', 'jsp', 'updated javascripts', 'programming', 'spring framework', 'design', 'finding bugs', 'sql server', 'training', 'hibernate', 'oracle', 'spring', 'processing', 'html', 'database', 'park', 'c++', 'created hibernate']
gap_list ['testing', 'information technology', 'web-based java', 'engineering', 'research', 'support', 'provisions', 'java developers', 'successful technology', 'creates design documents', "bachelor's degree", 'communications skills', 'v\n     ', 'develops', 'this', 'ejb 3.0', 'investigation', 'm/f/d', 'potential', 'contract', 'government integrator', 'jsf', 'de

skill word in JD: ['sql', 'relational database', 'testing', 'ejb3', 'jasper', 'web-based java', 'data normalization', 'engineering', 'research', 'job specifications', 'substituted', 'java', 'javascript', 'creates design documents', 'design patterns', 'orm software', 'dod secret', 'web services', 'jdo', 'ajax', 'web service', 'distributed', 'develops', 'agile', 'jdbc', 'computer science', 'building', 'bachelors degree', 'ejb 3.0', 'jsp', 'investigation', 'saas', 'sql server', 'jboss', 'position description', 'hibernate', 'jsf', 'principle', 'javaserver faces', 'designs', 'oracle', 'service', 'iterative development', 'html', 'methodologies', 'debugging']


127.0.0.1 - - [28/Sep/2020 05:55:29] "POST /get_gap/ HTTP/1.1" 200 -


skill word in CV: ['sql', 'j2ee', 'servlets', 'data analysis', 'analyzing', 'records', 'java', 'eap.java', 'javascript', 'testing and', 'j2ee designs', 'ajax', 'javascripts', 'data', 'jsps', 'bachelors', 'reviewing', 'software development', 'experience.•', 'jdbc', 'agile', 'computer science', 'masters degree', 'object oriented design', 'r', 'communication skills', 'taking', 'class diagrams', 'received', 'mysql', 'jsp', 'updated javascripts', 'programming', 'spring framework', 'design', 'finding bugs', 'sql server', 'training', 'hibernate', 'oracle', 'spring', 'processing', 'html', 'database', 'park', 'c++', 'created hibernate']
gap_list ['relational database', 'testing', 'ejb3', 'jasper', 'web-based java', 'data normalization', 'engineering', 'research', 'job specifications', 'substituted', 'creates design documents', 'design patterns', 'orm software', 'dod secret', 'web services', 'jdo', 'web service', 'distributed', 'develops', 'building', 'bachelors degree', 'ejb 3.0', 'investigatio